# 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). Для тех, кто выполнил задание с Росконтролем - напишите запрос для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного (то есть цифра вводится одна, а запрос проверяет оба поля)

In [2]:
import re
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup, Tag
import requests
from pymongo import MongoClient, TEXT
import string

In [12]:
client = MongoClient('localhost', 27017)
db = client['vacancies']    # database
params = db. params       # collection
vacancies= db. vacancies           # collection


In [4]:
def file_vacancies(title: str):
    vacancies_on_hh = file_vacancies_hh(title)
    vacancies_on_sjob = file_vacancies_sjob(title)

    vacancies_on_hh.extend(vacancies_on_sjob)
    df = pd.DataFrame(vacancies_on_hh)
    df.to_csv('vacancies.csv', index=False)

    with open('vacancies.csv', 'r') as f:
        if f.readable():
            print('\n вакансии записаны ')
        else:
            print('\n вакансии не записаны')

In [5]:
def file_vacancies_hh(title_: str):
    base_url = 'https://hh.ru/search/vacancy'
    headers = {

        'User-Agent':
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,  Gecko) Chrome/98.0.4758.82 Safari/537.36','Accept-Charset': 'utf-8',
        'Accept-Language': 'ru'
    }
    params = {
        'st': 'searchVacancy',
        'text': title_,
        'salary': '',
        'currency_code': 'RUR',
        'experience': 'noExperience',
        'employment': ['full', 'part', 'probation'],
        'schedule': ['fullDay', 'flexible', 'remote'],
        'order_by': 'relevance',
        'search_period': '0',
        'items_on_page': '20',
        'no_magic': 'true',
        'L_save_area': 'true',
        'page': '1'
    }

    print(f'\nЗапрос на {base_url}')
    response = requests.get(url=base_url, headers=headers, params=params)
    if not response.ok:
        print(f'Получен ответ сервера  на запрос')
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # Определbv количества страниц 
    vacancies_count = str.replace(soup.find('h1', attrs={'data-qa': 'bloko-header-3'}).text, '\xa0', '')
    vacancies_count = int(str.split(vacancies_count, ' ')[0])
    page_count = vacancies_count // 20 + 1 if vacancies_count % 20 > 0 else 0
    print(f'vacancies_count = {vacancies_count}', f'page_count = {page_count}')

    vacancies = []
    
    for i in range(0, page_count):
        params['page'] = i
        print(f'Запрос на {base_url}, page={i}')
        response = requests.get(url=base_url, headers=headers, params=params)
        if not response.ok:
            print(f'Получен ответ сервера  на запрос ')
                  
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        for item in soup.select('div.vacancy-serp-item'):
            title = item.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'}).text
            url = item.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
            city = item.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-address'})
            date_publication = item.find('span', attrs={'class': 'vacancy-serp-item__publication-date_long'})

            
            if item.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}) is not None:
                currency_str = item.find('span', attrs={'data-qa': 'vacancy-serp__vacancy-compensation'}).text
                salary_str = currency_str.replace('\xa0', '').replace('\u202f', '')  #  пробел

                salary, salary_max, salary_min = 0, 0, 0
                if 'от' in salary_str:
                    salary_min = int(salary_str.split(' ')[1])
                elif 'до' in salary_str:
                    salary_max = int(salary_str.split(' ')[1])
                elif '–' in salary_str:
                    temp_list = salary_str.split('–')
                    salary_min = int(temp_list[0].split(' ')[0])
                    salary_max = int(temp_list[1].split(' ')[1])
                else:
                    salary = int(salary_str.split(' ')[0])

                currency = currency_str.split(' ')[-1]
            else:
                salary, salary_max, salary_min = 0, 0, 0
                currency = None

            
            if item.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}) is not None:
                responsibility = item.find('div',
                                           attrs={'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'}).text
            else:
                responsibility = 'Не указаны'

            
            if item.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}) is not None:
                requirement = item.find('div', attrs={'data-qa': 'vacancy-serp__vacancy_snippet_requirement'}).text
            else:
                requirement = 'Не указаны'

            vacancies.append(
                {
                    'title': title,
                    'site': 'https://hh.ru/',
                    'url': url,
                    'salary': salary,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'currency': currency,
                    'city': city,
                    'responsibility': responsibility,
                    'requirement': requirement,
                    'date_publication': date_publication
                })
    
    print(f'В результате hh.ru получено {len(vacancies)} вакансий')
    return vacancies


def file_vacancies_sjob(title_: str):
    base_url = 'https://russia.superjob.ru/vacancy/search/'
    headers = {

        'User-Agent':
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,  Gecko) Chrome/98.0.4758.82 Safari/537.36','Accept-Charset': 'utf-8',
        'Accept-Language': 'ru'
    }
    params = {
        'keywords': title_,
        'page': ''
    }
    months = { '01': 'января', '02': 'февраля', '03': 'марта','04': 'апреля','05': 'мая','06': 'июня','07': 'июля','08': 'августа','09': 'сентября','10': 'октября','11': 'ноября','12': 'декабря'}
        
              
        
    dt = datetime.datetime.now()

    print(f'\nЗапрос на {base_url}')
    response = requests.get(url=base_url, headers=headers, params=params)
    if not response.ok:
        print(f'Получен ответ сервера  на запрос ')
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    #  количества страниц 
    vacancies_count = soup.select('h2 span')
    if len(vacancies_count) > 0:
        vacancies_count = int(vacancies_count[0].text.split(' ')[1].replace('\xa0', '').replace('\u202f', ''))
    else:
        vacancies_count = 10
    page_count = vacancies_count
    print(f'vacancies_count = {vacancies_count}', f'page_count = {page_count}')

    vacancies = []
   
    for i in range(1, page_count + 1):
        params['page'] = i
        print(f'Запрос на {base_url}, page={i}')
        response = requests.get(url=base_url, headers=headers, params=params)
        if not response.ok:
            print(f'Получен ответ сервера  на запрос \n '
                  f'file_vacancies_sjob()')
            break

        soup = BeautifulSoup(response.text, 'html.parser')

        for item in soup.select('div.f-test-vacancy-item'):
            title = item.find('div._2g1F- div div div a')
            #url = item.find('a', attrs={'data-qa': 'vacancy-serp__vacancy-title'})['href']
            #url = 'https://www.superjob.ru' [0]['href']
            link = 'https://www.superjob.ru/vacancy/search/'
            city = item.select('span.f-test-text-company-item-location span')[0].text

            
            soup_part = item.select('div.HSTPm span')[0]
            if len(soup_part.select('div')):
                soup_part.div.extract()
            requirement = ''.join(
                [i.text if isinstance(i, Tag) else i for i in soup_part.select('br')[0].next_siblings])
            responsibility = ''.join(
                [i.text if isinstance(i, Tag) else i for i in soup_part.select('br')[0].previous_siblings])

            
            #  приводим время 
            date_publication = item.select('span.f-test-text-company-item-location span')[0].text
            if date_publication == 'Вчера':
                date_publication = str(int(dt.strftime('%d')) - 1) + ' ' + months[dt.strftime('%m')]
            elif ':' in date_publication:
                date_publication = str(int(dt.strftime('%d'))) + ' ' + months[dt.strftime('%m')]

            # salary
            currency_str = item.select('span.f-test-text-company-item-salary span')[0].text
            salary_str = currency_str.replace('\xa0', '').replace('\u202f', '')  
            salary, salary_max, salary_min = 0, 0, 0
            if 'от' in salary_str:
                salary_min = int(re.sub('\D', '', salary_str))
            elif 'до' in salary_str and re.match('\d', salary_str[2:]) is not None:
                salary_max = int(re.sub('\D', '', salary_str))
            elif '—' in salary_str:
                temp_list = salary_str.split('—')
                salary_min = int(re.sub('\D', '', temp_list[0]))
                salary_max = int(re.sub('\D', '', temp_list[1]))
            elif re.match('\d', salary_str) is not None:
                salary = int(re.sub('\D', '', salary_str))
            else:
                salary = salary_str

            # currency
            if re.match('\d', salary_str) or re.match('\d', salary_str[2:]):
                currency = currency_str.split('\xa0')[-1]
            else:
                currency = None

            vacancies.append(
                {
                    'title': title,
                    'site': 'https://www.superjob.ru/',
                    #'url': url,
                    'url': link,
                    'salary': salary,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'currency': currency,
                    'city': city,
                    'responsibility': responsibility,
                    'requirement': requirement,
                    'date_publication': date_publication
                })

    
    print(f'В результате  superjob.ru получено {len(vacancies)} вакансий')
    return vacancies


file_vacancies('разнорабочий')


Запрос на https://hh.ru/search/vacancy
vacancies_count = 3627 page_count = 182
Запрос на https://hh.ru/search/vacancy, page=0
Запрос на https://hh.ru/search/vacancy, page=1
Запрос на https://hh.ru/search/vacancy, page=2
Запрос на https://hh.ru/search/vacancy, page=3
Запрос на https://hh.ru/search/vacancy, page=4
Запрос на https://hh.ru/search/vacancy, page=5
Запрос на https://hh.ru/search/vacancy, page=6
Запрос на https://hh.ru/search/vacancy, page=7
Запрос на https://hh.ru/search/vacancy, page=8
Запрос на https://hh.ru/search/vacancy, page=9
Запрос на https://hh.ru/search/vacancy, page=10
Запрос на https://hh.ru/search/vacancy, page=11
Запрос на https://hh.ru/search/vacancy, page=12
Запрос на https://hh.ru/search/vacancy, page=13
Запрос на https://hh.ru/search/vacancy, page=14
Запрос на https://hh.ru/search/vacancy, page=15
Запрос на https://hh.ru/search/vacancy, page=16
Запрос на https://hh.ru/search/vacancy, page=17
Запрос на https://hh.ru/search/vacancy, page=18
Запрос на https://

In [7]:
def db_plus_vacancies(client_: MongoClient, data_):
    
    
    db = client_['vacancies']
    vacancies = db['vacancies']

    counter = 0
    for item in data_:
        
        test = vacancies.find_one({'url': item['url']})

        if test is None:
            vacancies.insert_one(item)
            counter += 1

    print(f'Из {len(data_)} вакансий переданных в db_insert_vacancy()\n\t{counter} добавлено в базу данных'
          f'\n\t{len(data_)-counter} совпадают с имеющимися в базе данных')


In [70]:
def db_vacancies(client: MongoClient, title ='', city ='', salary =0, salary_label =''):
   
    
    assert salary_label in ['', 'more', 'less', 'equ'], f"salary_label_most be in ['more', 'less', 'equ'], " \
                                                        f"but received {salary_sign}"
    

    db = client['vacancies']
    vacancy = db['vacancies']

    
    list_params = []
    
    if salary != 0:
        if salary_label == 'more':
            list_params.append({'$or': [{'salary': {'$gte': salary}},
                                        {'salary_max': {'$gte': salary}},
                                        {'salary_min': {'$gte': salary}}]
                                })
        elif salary_label == 'less':
            list_params.append({'$or': [{'salary': {'$lte': salary}},
                                        {'salary_max': {'$lte': salary}},
                                        {'salary_min': {'$lte': salary}}]
                                })
        elif salary_label == 'equ' or salary_sign == '':
            list_params.append({'$or': [{'salary': {'$eq': salary}},
                                        {'$and': [{'salary_max': {'$gte': salary}},
                                                  {'salary_min': {'$lte': salary}}]
                                         }]
                                })
    if len(list_params) > 1:
       
        result = vacancies()
    else:
        result = vacancies()
    return result



vacancies_title = 'разнорабочий'
vacancies = file_vacancies_hh('разнорабочий')
vacancies.extend(file_vacancies_sjob('разнорабочий'))






if db_vacancies(client, salary=30000, salary_label ='more') > 30000:
    for item in db_vacancies(client, salary=30000, salary_label ='more'):
        pprint.pprint(item)
else:
    print(f'По запросу в БД ничего не найдено...')




Запрос на https://hh.ru/search/vacancy
vacancies_count = 3635 page_count = 182
Запрос на https://hh.ru/search/vacancy, page=0
Запрос на https://hh.ru/search/vacancy, page=1
Запрос на https://hh.ru/search/vacancy, page=2
Запрос на https://hh.ru/search/vacancy, page=3
Запрос на https://hh.ru/search/vacancy, page=4
Запрос на https://hh.ru/search/vacancy, page=5
Запрос на https://hh.ru/search/vacancy, page=6
Запрос на https://hh.ru/search/vacancy, page=7
Запрос на https://hh.ru/search/vacancy, page=8
Запрос на https://hh.ru/search/vacancy, page=9
Запрос на https://hh.ru/search/vacancy, page=10
Запрос на https://hh.ru/search/vacancy, page=11
Запрос на https://hh.ru/search/vacancy, page=12
Запрос на https://hh.ru/search/vacancy, page=13
Запрос на https://hh.ru/search/vacancy, page=14
Запрос на https://hh.ru/search/vacancy, page=15
Запрос на https://hh.ru/search/vacancy, page=16
Запрос на https://hh.ru/search/vacancy, page=17
Запрос на https://hh.ru/search/vacancy, page=18
Запрос на https://

TypeError: 'list' object is not callable